In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:

from solar_power.environment_prediction_model import SolarPredictionModelExtended 
from solar_power.solar_prediction_model import FeatureSolarPredictionModel
from electrical_load.load_prediction_model import BuildingEnergyModel

# * 모델 선언
# * 랜덤 시드 값을 모델 간 동일하게 42로 설정
# * 테스트 데이터 비율을 모델 간 동일하게 0.2로 설정

random_state = 42
test_size = 0.2

# * 기상 데이터 to 환경 감시 데이터 예측 모델
env_pred_model = SolarPredictionModelExtended("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
env_pred_model.load_data()
env_pred_model.hyperparameter_tuning()
env_pred_model.train_model()

# * 환경 감시 데이터 to 태양광 발전량 예측 모델
solar_pred_model = FeatureSolarPredictionModel("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
solar_pred_model.load_data()
solar_pred_model.feature_selection()
solar_pred_model.train_model()

# * 건물 전기 부하 예측 모델
energy_pred_model = BuildingEnergyModel("./electrical_load/merged_data.csv", random_state=random_state, test_size=test_size)
energy_pred_model.load_data()
energy_pred_model.train_model()

In [5]:
# * 태양광 발전량 예측 모델에 입력할 정규화된 입력 데이터
import numpy as np
rand_int = np.random.randint(0, len(env_pred_model.X_test))

test_x = env_pred_model.X_test.iloc[rand_int:rand_int+1, :]
test_x

,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
439,0.3,0.8,1.0,0.571429,0.833333,0.265264,0.290323,0.652174


In [6]:
# * 예측한 태양광 발전량
res_solar = env_pred_model.predict(test_x) # 기상 데이터 -> 환경감시 데이터
res_solar = solar_pred_model.predict(res_solar) # 환경감시 데이터 -> 태양광 발전량 데이터
res_solar

array([8.99885641])

In [7]:
# * 실제 태양광 발전량
solar_pred_model.y_test.iloc[rand_int:rand_int+1, :]

,축구장,학생회관,중앙창고,학사과정,다산빌딩,시설관리동,대학C동,동물실험동,중앙도서관,LG도서관,신재생에너지동,삼성환경동,중앙연구기기센터,산업협력관,기숙사 B동
439,2.2,1.5,0.9,0.9,0.8,2.2,0.7,1.6,1.0,1.2,3.0,2.0,3.7,4.8,2.0


In [8]:
# * 부하량 예측 모델에 입력할 정규화된 입력 데이터
test_x = energy_pred_model.X_test.iloc[rand_int:rand_int+1, :]
test_x

,time,1시간기온,1시간강수량,일최고기온,일최저기온,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
439,7,25.0,0.0,31.0,24.0,False,False,False,False,False,True,False


In [9]:
# * 예측한 전기 부하량
res_load = energy_pred_model.predict(test_x)
res_load

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
0,120.7,429.8,413.9,482.4,138.1,263.5,32.8,57.5,87.8,49.8,...,36.2,30.2,27.6,32.3,32.5,22.0,51.1,19.0,47.6,21.5


In [10]:
# * 실제 전기 부하량
energy_pred_model.y_test.iloc[rand_int:rand_int+1, :]

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
439,113.5,457.7,403.2,489.8,125.6,285.7,27.3,53.3,214.5,138.3,...,39.3,23.3,36.6,34.0,31.6,33.4,49.1,19.4,46.7,20.5


In [19]:
# * 예측 발전량과 부하량을 통해 전기 요금 산출
total_predicted_solar = res_solar.sum().sum()
total_predicted_load = res_load.sum().sum()

# print("Predicted solar power: ", total_predicted_solar)
# print("Predicted load: ", total_predicted_load)


6930.5